In [1]:
import argparse, sys, os, random, string
from typing import Optional, Union
from dataclasses import dataclass
# import logging

import numpy as np
import pandas as pd
import torch
from torch.utils.data import Dataset

from sklearn.model_selection import KFold

import transformers
from transformers import AutoModelForMultipleChoice, TrainingArguments, Trainer
from transformers.tokenization_utils_base import PreTrainedTokenizerBase, PaddingStrategy
# !pip install sentencepiece
# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
from kobert_tokenizer import KoBERTTokenizer

import wandb
## run `wandb login --relogin`` in TERMINAL if you want to use your own  wandb profile


print(f"unsing {torch.device('cuda' if torch.cuda.is_available() else 'cpu')} with {torch.__version__}")

/usr/local/lib/python3.8/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


unsing cuda with 1.10.0+cu111


In [2]:
class TextDataset(Dataset):
    def __init__(self, data: pd.DataFrame, tokenizer, indices=None):
        
        if indices is None:     ## when testing, not training, use the whole data without sampling indices
            indices = range(len(data))
            
        self.data = data.iloc[indices]
        self.hyps = self.data.filter(regex='hyp').keys()
        self.tokenizer=tokenizer
        
    def __len__(self)  :
        return len(self.data)
    
    def __getitem__(self, idx):
        # obs1 sentences: questions that work as given sentence for multiple choice
        obs1_sentences = [self.data.iloc[idx]['OBS1']] * len(self.hyps)
        # hyp_obs2 sentences: option for multiple choices. obs2 sentece is fixed according to obs1 sentence. only hyppthesis sentences differs.
        obs2_sentence = self.data.iloc[idx]['OBS2']
        hyp_obs2_sentences = [f"{self.data.iloc[idx][hyp]} {obs2_sentence}" for hyp in self.hyps]

        # text-encoding (tokenizing)
        text_embed = self.tokenizer(obs1_sentences, hyp_obs2_sentences, padding=True, return_tensors='pt')
        input_ids = text_embed['input_ids']
        token_type_ids = text_embed['token_type_ids']
        attention_mask = text_embed['attention_mask']
        
        # answers - 0, 1, 2, ..
        label = torch.tensor(self.data.iloc[idx]['label']).unsqueeze(0)
        
        return {'input_ids' : input_ids, 
                'token_type_ids':token_type_ids, 
                'attention_mask' : attention_mask, 
                'labels':label } 
    

@dataclass
class DataCollatorForMultipleChoice:
    """
    Data collator that will dynamically pad the inputs for multiple choice received.
    """
    tokenizer: PreTrainedTokenizerBase 
    padding: Union[bool, str, PaddingStrategy] = True
    max_length: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None

    def __call__(self, features):
        label_name = "label" if "label" in features[0].keys() else "labels"
        labels = [feature.pop(label_name) for feature in features]
        batch_size = len(features)
        num_choices = len(features[0]["input_ids"])

        # flattnen all the inputs/attetions masks etc.
        flattened_features = [[{k: v[i] for k, v in feature.items()} for i in range(num_choices)] for feature in features]
        flattened_features = sum(flattened_features, [])
        
        # This will return a dictionary with tensors of shape `(batch_size * 4) x seq_length`
        batch = self.tokenizer.pad(
                flattened_features,
                padding=self.padding,
                max_length=self.max_length,
                pad_to_multiple_of=self.pad_to_multiple_of,
                return_tensors="pt"
                )
        
        # Un-flatten
        batch = {k: v.view(batch_size, num_choices, -1) for k, v in batch.items()}
        # Add back labels
        batch["labels"] = torch.tensor(labels, dtype=torch.int64)
        return batch



def get_tokenizer(text_model: str):
    if "kobert" in text_model:
        # !pip install sentencepiece
        # !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
        from kobert_tokenizer import KoBERTTokenizer
        tokenizer = KoBERTTokenizer.from_pretrained(text_model)
    elif "klue" in text_model:
        from transformers import AutoTokenizer
        tokenizer = AutoTokenizer.from_pretrained(text_model, use_fast=True)
    else:
        print("Got unexpected text model and load AutoTokenizer. Please Check the tokenizer model")
        tokenizer = AutoTokenizer.from_pretrained(text_model, use_fast=True)
    return tokenizer

In [3]:
class KFoldTrainer :
    def __init__(self, train_set: pd.DataFrame, test_set:pd.DataFrame, fold_idx: int, tokenizer, configs, args) :
        self.train_set = train_set
        self.test_set = test_set
        self.args = args
        
        self.model = configs['text_model']
        self.tokenizer = tokenizer
        self.data_collator = DataCollatorForMultipleChoice(tokenizer)


        self.k = configs['kfolds']
        self.kfold_split = list(KFold(n_splits=self.k, shuffle=True, random_state=42).split(train_set))
        train_idx, val_idx = self.kfold_split[fold_idx]
        
        self.dataset_dict = {'train': TextDataset(self.train_set, self.tokenizer, train_idx), 
                       'valid' : TextDataset(self.train_set, self.tokenizer, val_idx),
                       'test': TextDataset(self.test_set, self.tokenizer)}        

        self.model = AutoModelForMultipleChoice.from_pretrained(self.model)
        
        wandb.watch(self.model)
        wandb.config.update(self.args)        
        
        self.trainer = Trainer(
            model = self.model,
            args = self.args,
            train_dataset = self.dataset_dict['train'],
            eval_dataset = self.dataset_dict['valid'],
            data_collator = self.data_collator,
            compute_metrics = self.compute_metrics,  
            )        
        
        
    def compute_metrics(self, eval_predictions : transformers.EvalPrediction):
        predictions, label_ids = eval_predictions
        preds = np.argmax(predictions, axis=1)
        return {"accuracy": (preds == label_ids).astype(np.float32).mean().item()}

          
    def train_fold(self, fold_idx):
        # logging.info(f'{fold}/{self.k} - fold started')
        print(f'===== {fold_idx+1}/{self.k} - fold TRAINING started =====')

        self.model.train()
        self.trainer.train()
        
        print(f'===== {fold_idx+1}/{self.k} - fold TESTING started =====')
        self.model.eval()
        metrics = self.trainer.evaluate(self.dataset_dict['test'])
        print(f'== {fold_idx+1}th fold metric is {metrics} ==')
                
        del self.dataset_dict       ## re-sample datasets with another k-fold indices
        torch.cuda.empty_cache()    ## empty CUDA memory before starting next fold

In [4]:
## !! CHECK !! the output_dir in main() function !!
configs = {
    'text_model' : 'klue/roberta-large',
    'kfolds' : 3,
    'train_path' : '/kovar-vol/kovar/dataset/train.json',
    'test_path':'/kovar-vol/kovar/dataset/photo_test.json'
}


args = TrainingArguments(
    output_dir = f"{configs['text_model']}-ft_base",    ## this is just fundamental output_dir. It should change at main() to prevent overwrite
    overwrite_output_dir = False,
    evaluation_strategy = "epoch", #evaluation is done (and logged) every eval_steps
    save_strategy='epoch',
    load_best_model_at_end=True,
    save_total_limit=2,
    logging_strategy = "steps", #logging is done every logging steps
    learning_rate = 1e-6,
    logging_steps = 500, #number of update steps between two logs if logging_strategy = "steps"
    eval_steps = 500,
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    num_train_epochs = 5, 
    report_to = "wandb",
    weight_decay = 0.01, #The weight decay to apply (if not zero) to all layers except all bias and LayerNorm weights in AdamW optimizer.
)

In [5]:
def main():
    # Load datasets   
    train_set = pd.read_json(configs['train_path'], lines=True)
    test_set = pd.read_json(configs['test_path'], lines=True)

    # Load Tokenizer
    tokenizer = get_tokenizer(configs['text_model'])
    
    wandb.init(project='sage-sky-3')
    
    ## !! 혹 중간에 학습이 끊기면, ~ in [2,3] 이런 식으로 fold_idx를 조정해서 학습을 이어가자 !!
    for fold_idx in [2]:
        torch.cuda.empty_cache()   
        args.output_dir = f"/kovar-vol/kovar/models/klueRoBERTa_ft/klueROBERTa_ft-fold_{fold_idx+1}"
        kfold_trainer = KFoldTrainer(train_set, test_set, fold_idx, tokenizer, configs, args)
        kfold_trainer.train_fold(fold_idx)
        
    wandb.finish()
    

In [6]:
if __name__ == '__main__':
    main()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: rkfcl226 (kovar). Use `wandb login --relogin` to force relogin


Some weights of the model checkpoint at klue/roberta-large were not used when initializing RobertaForMultipleChoice: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.bias', 'lm_head.decoder.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForMultipleChoice from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForMultipleChoice were not initialized from the model checkpoint at klue/roberta-large and are newly initialized: ['classifier.bias', 'roberta.pooler.dense.weight', 'roberta.pooler.dense.bias', 'classifier.we

===== 3/3 - fold TRAINING started =====


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.385426,0.856330
2,0.672700,0.302593,0.888046
3,0.367900,0.275785,0.892383
4,0.301500,0.268533,0.897804
5,0.265200,0.267413,0.897262


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '
/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqu

===== 3/3 - fold TESTING started =====


/usr/local/lib/python3.8/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


== 3th fold metric is {'eval_loss': 0.24344861507415771, 'eval_accuracy': 0.9235197305679321, 'eval_runtime': 95.4465, 'eval_samples_per_second': 12.74, 'eval_steps_per_second': 0.796, 'epoch': 5.0} ==


eval/accuracy,▁▄▅▅▅█
eval/loss,█▄▃▂▂▁
eval/runtime,▅████▁
eval/samples_per_second,█▁▁▁▁▁
eval/steps_per_second,█▁▁▁▁▁
train/epoch,▁▁▃▃▅▅▆▇███
train/global_step,▁▁▃▃▅▅▆▇███
train/learning_rate,█▆▃▁
train/loss,█▃▂▁
train/total_flos,▁
train/train_loss,▁
